# Import Packages

In [1]:
%pylab inline
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import itertools
import pickle
from datetime import datetime
from itertools import chain
import itertools
import json
from pyspark.sql.functions import desc, explode
import datetime
from collections import defaultdict
from operator import itemgetter
import yaml
import os
credentials = yaml.load(open(os.path.expanduser('plotlyCred.yml')))
plotly.tools.set_credentials_file(credentials.get('username'), api_key=credentials.get('api_key'))

Populating the interactive namespace from numpy and matplotlib


# Load Parquet Files from S3

In [2]:
topic_by_category = spark.read.parquet("s3a://meetupdf/topic_by_category")

In [3]:
urlkey =spark.read.parquet("s3a://meetupdf/urlkey")

In [4]:
topic_by_category.show(5)

+-------+-------------+----------------+--------------------+
|     id|         city|        category|              topics|
+-------+-------------+----------------+--------------------+
| 115511|     Portland|religion/beliefs|[Witches, Pagan, ...|
|9593962|      Houston|     socializing|[Adult/Teens with...|
| 432739|     Oak Park| career/business|[Business, Leader...|
| 986420|San Francisco|            LGBT|[Gay Dads, Gay Pa...|
|  88008|     New York|  hobbies/crafts|[Jewelry Making, ...|
+-------+-------------+----------------+--------------------+
only showing top 5 rows



In [5]:
urlkey.show(5)

+--------+------------+-------------+-------------+--------------------+--------------------+
|group_id|    event_id|   group_city|   event_time|               topic|              urlkey|
+--------+------------+-------------+-------------+--------------------+--------------------+
|18649748|   238062438|    San Diego|1489511700000|[[Entrepreneurshi...|[entrepreneurship...|
|19527815|vvdsxlywfbvb|Saint Charles|1489705200000|[[Robotics,roboti...|[robotics, electr...|
|18788220|   238052232|       London|1489170600000|[[Hiking,hiking],...|[hiking, singles-...|
|20379974|fjqmbmywgbjb|    Vancouver|1491530400000|[[Wine,wine], [Di...|[wine, diningout,...|
| 4369102|   237176191|       Kanata|1489847400000|[[Spirit Guides,s...|[spirit-guides, s...|
+--------+------------+-------------+-------------+--------------------+--------------------+
only showing top 5 rows



In [6]:
# tech_groups = topic_by_category.filter(topic_by_category['category'] == "tech")

# Convert to Pandas

In [7]:
def convert_pd_datetime(df):
	'''input: a sparksql dataframe
		output: a panda dataframe that has converted timestamp to datetime '''
	urlkey_df = df.toPandas()
	urlkey_df['datetime'] = pd.to_datetime(urlkey_df['event_time'],unit = 'ms')
	urlkey_df['date'] =  pd.DatetimeIndex(urlkey_df['datetime']).date
	return urlkey_df

In [8]:
topic_time_panda = convert_pd_datetime(urlkey)
topic_category_panda = topic_by_category.toPandas()
topic_time_panda = topic_time_panda.dropna()

In [9]:
# Category Distribution
data = Data([Histogram(x=topic_category_panda['category'])])
py.iplot(data)

In [11]:
def unique_urlkey(df):
    return df['urlkey'].unique()
def top_urlkey_count(df):
    urlkey_count = df.groupby('urlkey')['group_id'].count()
    urlkey_count = pd.DataFrame(urlkey_count)
    return urlkey_count

In [12]:
topic_category_panda.head(5)

,id,city,category,topics
0,115511,Portland,religion/beliefs,"[Witches, Pagan, Asatru, Goddess, Shamanism, D..."
1,9593962,Houston,socializing,"[Adult/Teens with Asperger/Spectrum Disorders,..."
2,432739,Oak Park,career/business,"[Business, Leadership, Professional Networking..."
3,986420,San Francisco,LGBT,"[Gay Dads, Gay Parents]"
4,88008,New York,hobbies/crafts,"[Jewelry Making, Metal clay, Bead Addicts, Mak..."


In [13]:
topic_time_panda.head(5)

,group_id,event_id,group_city,event_time,topic,urlkey,datetime,date
0,18649748.0,238062438,San Diego,1.489512e+12,"[(Entrepreneurship, entrepreneurship), (Entrep...","[entrepreneurship, business-entrepreneur-netwo...",2017-03-14 17:15:00,2017-03-14
1,19527815.0,vvdsxlywfbvb,Saint Charles,1.489705e+12,"[(Robotics, robotics), (Electronics, electroni...","[robotics, electronics, newtech, microcontroll...",2017-03-16 23:00:00,2017-03-16
2,18788220.0,238052232,London,1.489171e+12,"[(Hiking, hiking), (Singles Who Love to Travel...","[hiking, singles-who-love-to-travel-travel, bu...",2017-03-10 18:30:00,2017-03-10
3,20379974.0,fjqmbmywgbjb,Vancouver,1.491530e+12,"[(Wine, wine), (Dining Out, diningout), (Night...","[wine, diningout, nightlife, socialnetwork, ho...",2017-04-07 02:00:00,2017-04-07
4,4369102.0,237176191,Kanata,1.489847e+12,"[(Spirit Guides, spirit-guides), (Self Explora...","[spirit-guides, self-exploration, transformati...",2017-03-18 14:30:00,2017-03-18


In [14]:
category_topic = topic_category_panda[['category','topics']]
category_topic.head(5)

,category,topics
0,religion/beliefs,"[Witches, Pagan, Asatru, Goddess, Shamanism, D..."
1,socializing,"[Adult/Teens with Asperger/Spectrum Disorders,..."
2,career/business,"[Business, Leadership, Professional Networking..."
3,LGBT,"[Gay Dads, Gay Parents]"
4,hobbies/crafts,"[Jewelry Making, Metal clay, Bead Addicts, Mak..."


# Manipulate Data

In [15]:
def key_list(l):
    merged_list = list(itertools.chain(*l))
    merged_list = [word.strip(" ").lower() for word in merged_list]
    return merged_list
def category_topic_dict(df):
    category_double_lists = defaultdict(list)
    for i in range(len(df)):
        category_double_lists[df['category'][i]].append(df['topics'][i])
    merged_list = {}
    for k,v in category_double_lists.items():
        merged_list[k] = key_list(v)
    return merged_list

In [16]:
top_5_category = pd.DataFrame(topic_category_panda['category'].value_counts()[:5])
top_5_category = top_5_category.reset_index()
top_5_category.columns = ['category','count']
top_5_category

,category,count
0,career/business,188
1,socializing,163
2,tech,133
3,language/ethnic identity,122
4,outdoors/adventure,92


In [17]:
def filter_dict(dic):
    top5cat = dict()
    for k,v in dic.items():
        if k in top_5_category['category'].tolist():
            top5cat[k] = v
    return top5cat

In [18]:
category_list_topics = category_topic_dict(category_topic)
top5cat = filter_dict(category_list_topics)

# Label group category by topics

In [19]:
def label(df,dic):
    for (k,v) in dic.items():
        df[k] = df['urlkey'].apply(lambda x: [ True if word in v else False for word in x ].count(True) )
    return df

In [20]:
test = label(topic_time_panda,top5cat)
test = test.reset_index()

In [21]:
test.head(5)

,index,group_id,event_id,group_city,event_time,topic,urlkey,datetime,date,career/business,outdoors/adventure,language/ethnic identity,tech,socializing
0,0,18649748.0,238062438,San Diego,1.489512e+12,"[(Entrepreneurship, entrepreneurship), (Entrep...","[entrepreneurship, business-entrepreneur-netwo...",2017-03-14 17:15:00,2017-03-14,4,1,2,2,3
1,1,19527815.0,vvdsxlywfbvb,Saint Charles,1.489705e+12,"[(Robotics, robotics), (Electronics, electroni...","[robotics, electronics, newtech, microcontroll...",2017-03-16 23:00:00,2017-03-16,2,0,0,5,0
2,2,18788220.0,238052232,London,1.489171e+12,"[(Hiking, hiking), (Singles Who Love to Travel...","[hiking, singles-who-love-to-travel-travel, bu...",2017-03-10 18:30:00,2017-03-10,3,6,5,0,6
3,3,20379974.0,fjqmbmywgbjb,Vancouver,1.491530e+12,"[(Wine, wine), (Dining Out, diningout), (Night...","[wine, diningout, nightlife, socialnetwork, ho...",2017-04-07 02:00:00,2017-04-07,3,3,2,1,4
4,4,4369102.0,237176191,Kanata,1.489847e+12,"[(Spirit Guides, spirit-guides), (Self Explora...","[spirit-guides, self-exploration, transformati...",2017-03-18 14:30:00,2017-03-18,4,3,3,2,4


In [22]:
def label_by_count(df):
    labels = ['career/business','outdoors/adventure','language/ethnic identity','tech','socializing']
    final_label = []
    for i in range(len(test)):
        list_labels = [df[label][i] for label in labels]
        index, element = max(enumerate(list_labels), key=itemgetter(1))
        final_label.append(labels[index])
    return final_label
final_label = label_by_count(test)

In [23]:
test['label'] = pd.Series(final_label, index = test.index)

In [24]:
test.head(5)

,index,group_id,event_id,group_city,event_time,topic,urlkey,datetime,date,career/business,outdoors/adventure,language/ethnic identity,tech,socializing,label
0,0,18649748.0,238062438,San Diego,1.489512e+12,"[(Entrepreneurship, entrepreneurship), (Entrep...","[entrepreneurship, business-entrepreneur-netwo...",2017-03-14 17:15:00,2017-03-14,4,1,2,2,3,career/business
1,1,19527815.0,vvdsxlywfbvb,Saint Charles,1.489705e+12,"[(Robotics, robotics), (Electronics, electroni...","[robotics, electronics, newtech, microcontroll...",2017-03-16 23:00:00,2017-03-16,2,0,0,5,0,tech
2,2,18788220.0,238052232,London,1.489171e+12,"[(Hiking, hiking), (Singles Who Love to Travel...","[hiking, singles-who-love-to-travel-travel, bu...",2017-03-10 18:30:00,2017-03-10,3,6,5,0,6,outdoors/adventure
3,3,20379974.0,fjqmbmywgbjb,Vancouver,1.491530e+12,"[(Wine, wine), (Dining Out, diningout), (Night...","[wine, diningout, nightlife, socialnetwork, ho...",2017-04-07 02:00:00,2017-04-07,3,3,2,1,4,socializing
4,4,4369102.0,237176191,Kanata,1.489847e+12,"[(Spirit Guides, spirit-guides), (Self Explora...","[spirit-guides, self-exploration, transformati...",2017-03-18 14:30:00,2017-03-18,4,3,3,2,4,career/business


# Create TimeSeries 

In [25]:
def timeseries_topn_urlkey(df,column, n):
    """Input: DataFrame and N: number of urlkeys you want to plot
    Output: DataFrame that have top n values for everyday """
    #get urlkey count
    urlkey_count = df.groupby(column)['group_id'].count()
    urlkey_count = pd.DataFrame(urlkey_count)
    
    # sort urlkey count in descending order
    ordered_urlkey_count = urlkey_count.sort_values('group_id', ascending=False)
#     pop_topics_list = ordered_urlkey_count.index.values[:10].tolist()
    
    # get the top N values
    pop_urlkey_count = ordered_urlkey_count.index.values[:n].tolist()
    
    #create timeseries 
    new_topic_pd = df[[column,'date']]
    topic_index = new_topic_pd[column].isin(pop_urlkey_count)
    top_100_topic_pd = new_topic_pd[topic_index]
    
    #get urlkey count/day 
    crosstab_topic_date = pd.crosstab(top_100_topic_pd['date'],top_100_topic_pd[column])
    return crosstab_topic_date

# Top N Popular Topics

In [27]:
top6topics = timeseries_topn_urlkey(test,'label',6)

In [28]:
py.iplot([{
    'x': top6topics.index,
    'y': top6topics[col],
    'name': col
}  for col in top6topics.columns])

# Specific Topics to track

In [ ]:
def timeseries_specific_topics(df,spec_topics):
#     topic_count = df.groupby('topic')['group_id'].count()
#     topic_count = pd.DataFrame(topic_count)
#     ordered_topic_count = topic_count.sort_values('group_id', ascending=False)
    new_topic_pd = df[['topic','date']]
    topic_index = df['topic'].isin(spec_topics)
    top_100_topic_pd = new_topic_pd[topic_index]
    crosstab_topic_date = pd.crosstab(top_100_topic_pd['date'],top_100_topic_pd['topic'])
    return crosstab_topic_date

In [ ]:
datascience = ['Tech','Data Science','Web Development','Python', 'Data Analytics','Spark','JavaScript','Hadoop']
datasciencetopics = timeseries_specific_topics(df,datascience)

In [ ]:
py.iplot([{
    'x': datasciencetopics.index,
    'y': datasciencetopics[col],
    'name': col
}  for col in datasciencetopics.columns])